In [1]:
# imports
import pandas as pd
import numpy

In [2]:
# Read in CSV file to create EPA FRS dataframe
df_epa_frs = pd.read_csv("STATE_SINGLE_MN.csv")
df_epa_frs.head()

/var/folders/_9/80bxm26n3b5300q7n7g729_c0000gn/T/ipykernel_23700/3096208467.py:2: DtypeWarning: Columns (5,7,13,14,15,16,21,27,29,30,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_epa_frs = pd.read_csv("STATE_SINGLE_MN.csv")


,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,...,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110055452380,SHERWIN-WILLIAMS 3296,10690 BALTIMORE ST NE,NaN,10690 BALTIMORE ST NE,ANOKA,27003,MN,MINNESOTA,...,5231,"PAINT, GLASS, AND WALLPAPER STORES",45.16277,-93.23254,FRS-GEOCODE,ADDRESS MATCHING-HOUSE NUMBER,30.0,CENTER OF A FACILITY OR STATION,NAD83,NaN
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110055527835,PARKER AUTO MALL,6771 MAIN ST,NaN,6771 MAIN ST,CHISAGO,27025,MN,MINNESOTA,...,NaN,NaN,45.51152,-92.97108,FRS-GEOCODE,ADDRESS MATCHING-HOUSE NUMBER,30.0,CENTER OF A FACILITY OR STATION,NAD83,NaN
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110069113113,150TH STREET - AASTAD TOWNSHIP,SEE LOCATION DESCRIPTION,NaN,AASTAD TOWNSHIP,OTTER TAIL,27111,MN,MINNESOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAD83,NaN
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110068363425,AASTAD TOWNSHIP DUMP SITE,SEE LOCATION DESCRIPTION,NaN,AASTAD TOWNSHIP,OTTER TAIL,27111,MN,MINNESOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAD83,NaN
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110068290904,GARY T LIEN FARM,12701 220TH AVE,NaN,AASTAD TOWNSHIP,OTTER TAIL,27111,MN,MINNESOTA,...,NaN,NaN,46.14703,-96.02937,FRS-GEOCODE,ADDRESS MATCHING-HOUSE NUMBER,30.0,CENTER OF A FACILITY OR STATION,NAD83,NaN


In [3]:
# inspecting columns
df_epa_frs.columns

Index(['FRS_FACILITY_DETAIL_REPORT_URL', 'REGISTRY_ID', 'PRIMARY_NAME',
       'LOCATION_ADDRESS', 'SUPPLEMENTAL_LOCATION', 'CITY_NAME', 'COUNTY_NAME',
       'FIPS_CODE', 'STATE_CODE', 'STATE_NAME', 'COUNTRY_NAME', 'POSTAL_CODE',
       'FEDERAL_FACILITY_CODE', 'FEDERAL_AGENCY_NAME', 'TRIBAL_LAND_CODE',
       'TRIBAL_LAND_NAME', 'CONGRESSIONAL_DIST_NUM', 'CENSUS_BLOCK_CODE',
       'HUC_CODE', 'EPA_REGION_CODE', 'SITE_TYPE_NAME', 'LOCATION_DESCRIPTION',
       'CREATE_DATE', 'UPDATE_DATE', 'US_MEXICO_BORDER_IND', 'PGM_SYS_ACRNMS',
       'INTEREST_TYPES', 'NAICS_CODES', 'NAICS_CODE_DESCRIPTIONS', 'SIC_CODES',
       'SIC_CODE_DESCRIPTIONS', 'LATITUDE83', 'LONGITUDE83', 'CONVEYOR',
       'COLLECT_DESC', 'ACCURACY_VALUE', 'REF_POINT_DESC', 'HDATUM_DESC',
       'SOURCE_DESC'],
      dtype='object')

In [4]:
# create new dataframe dropping unneccessary columns
df_epa_frs_clean = df_epa_frs[['REGISTRY_ID', 'PRIMARY_NAME', 'LOCATION_ADDRESS', 'SITE_TYPE_NAME', 'INTEREST_TYPES', 'LATITUDE83', 'LONGITUDE83']]


In [5]:
#drop rows with null values in coordinate, interest types, location address, or site types columns
df_epa_frs_clean = df_epa_frs_clean.dropna(axis=0, subset=['LATITUDE83'])
df_epa_frs_clean = df_epa_frs_clean.dropna(axis=0, subset=['LONGITUDE83'])
df_epa_frs_clean = df_epa_frs_clean.dropna(axis=0, subset=['INTEREST_TYPES'])
df_epa_frs_clean = df_epa_frs_clean.dropna(axis=0, subset=['SITE_TYPE_NAME'])
df_epa_frs_clean = df_epa_frs_clean.dropna(axis=0, subset=['LOCATION_ADDRESS'])

In [6]:
df_epa_frs_clean.head()

,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SITE_TYPE_NAME,INTEREST_TYPES,LATITUDE83,LONGITUDE83
0,110055452380,SHERWIN-WILLIAMS 3296,10690 BALTIMORE ST NE,STATIONARY,"STATE MASTER, VSQG",45.16277,-93.23254
1,110055527835,PARKER AUTO MALL,6771 MAIN ST,STATIONARY,VSQG,45.51152,-92.97108
4,110068290904,GARY T LIEN FARM,12701 220TH AVE,STATIONARY,STATE MASTER,46.14703,-96.02937
5,110068936263,HAROLD BOESE FARM PBR,12388 COUNTY HIGHWAY 1,STATIONARY,STATE MASTER,46.14108,-96.10197
8,110068035726,VANHORN PROPERTY,18939 COUNTY HIGHWAY 26,STATIONARY,STATE MASTER,46.10847,-96.09263


In [7]:
# drop rows where interest type is "state master" only
df_epa_frs_clean = df_epa_frs_clean[df_epa_frs_clean.INTEREST_TYPES != "STATE MASTER"]


In [8]:
# Create smaller dataframes based on strings for different interest types to filter to relevant sites
df1 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("AIR EMISSION INVENTORY")]
df2 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("AIR MAJOR ")]
df3 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("AIR MINOR ")]
df4 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("AIR SYNTHETIC MINOR ")]
df5 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("ELECTRIC GENERATOR")]
df6 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("ELECTRIC POWER GENERATION")]
df7 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("ELECTRIC POWER GENERATOR")]
df8 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("ETHANOL FACILITY")]
df9 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("GASOLINE")]
df10 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("GREENHOUSE GAS")]
df11 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("ANIMAL")]
df12 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("LIVESTOCK")]
df13 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("TRI REPORTER")]
df14 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("TSCA SUBMITTER")]
df15 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("EPCRA")]
df16 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("FRP")]
df17 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("SPCC")]
df18 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("MINERAL EXPLORATION")]
df19 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("CATHODE RAY")]
df20 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("CESQG")]
df21 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("CORRECTIVE ACTION")]
df22 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("HAZARDOUS WASTE")]
df23 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("INCINERATOR")]
df24 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("INFECTIOUS WASTE")]
df25 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("LQG")]
df26 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("SQG")]
df27 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("TSD")]
df28 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("UIC")]
df29 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("USED OIL")]
df30 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("COMPLIANCE")]
df31 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("ENFORCEMENT")]
df32 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("PESTICIDE")]
df33 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("RAD NPL")]
df34 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("RAD WIPP")]
df35 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("RADIOACTIVE")]
df36 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("CLEANUP")]
df37 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("REMEDIATION")]
df38 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("BROWNFIELDS")]
df39 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("REMEDIAL")]
df40 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("SEDIMENTS")]
df41 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("SUPERFUND")]
df42 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("E-WASTE")]
df43 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("INDUSTRIAL SITES")]
df44 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("MATERIAL RECYCLING")]
df45 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("MINE")]
df46 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("MINING")]
df47 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("OIL CONTROL")]
df48 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("REFUSE DISPOSAL")]
df49 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("SCRAP TIRE MANAGEMENT")]
df50 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("SEWAGE SLUDGE")]
df51 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("SOLID WASTE")]
df52 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("LEAKING STORAGE TANK")]
df53 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("UNDERGROUND STORAGE TANK")]
df54 = df_epa_frs_clean[df_epa_frs_clean["INTEREST_TYPES"].str.contains("BIOSOLIDS")]


In [9]:
# create joined EPA FRS dataframe to bring together all of the string-filtered dfs from above 
final_epa_frs_clean = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,df31,df32,df33,df34,df35,df36,df37,df38,df39,df40,df41,df42,df43,df44,df45,df46,df47,df48,df49,df50,df51,df52,df53,df54], axis=0)

In [11]:
final_epa_frs_clean.tail(15)

,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SITE_TYPE_NAME,INTEREST_TYPES,LATITUDE83,LONGITUDE83
146806,110009401655,MET COUNCIL - ROSEMOUNT WWTP,4005 140TH STREET EAST,STATIONARY,"AIR EMISSIONS CLASSIFICATION UNKNOWN, BIOSOLID...",44.747222,-93.024167
151670,110000733591,SAINT JAMES,1305 WESTON AVE NO,STATIONARY,"BIOSOLIDS, COMMUNITY WATER SYSTEM, ICIS-NPDES ...",43.988204,-94.614913
156181,110000573941,MET COUNCIL METROPOLITAN WWTP,2400 CHILDS RD,STATIONARY,"AIR MAJOR, BIOSOLIDS, COMPLIANCE ACTIVITY, COM...",44.923243,-93.046563
158585,110000733582,SAINT PETER WWTP,400 WEST JULIEN ST,STATIONARY,"BIOSOLIDS, ICIS-NPDES MAJOR, ICIS-NPDES NON-MA...",44.338986,-93.943175
162209,110058356684,MET COUNCIL - BLUE LAKE WWTP,6957 COUNTY ROAD 101 E,STATIONARY,"AIR EMISSIONS CLASSIFICATION UNKNOWN, BIOSOLID...",44.800170,-93.428060
162542,110055254718,SMSC ORGANICS RECYCLING FACILITY,1905 CANTERBURY ROAD,STATIONARY,"BIOSOLIDS, ICIS-NPDES MINOR",44.767840,-93.469470
174405,110070003873,THIEF RIVER FALLS,803 BARZEN AVE SO,STATIONARY,"BIOSOLIDS, ICIS-NPDES NON-MAJOR",48.118130,-96.197750
178753,110070000304,VIRGINIA,1204 SOUTHERN DRIVE,STATIONARY,"BIOSOLIDS, ICIS-NPDES NON-MAJOR",47.508632,-92.547315
181878,110008815354,WASECA WWTP,35408 110TH ST,STATIONARY,"AIR EMISSIONS CLASSIFICATION UNKNOWN, BIOSOLID...",44.071759,-93.547038
187121,110000760105,WILLMAR,1400 SE SEVENTH ST.,STATIONARY,"BIOSOLIDS, FORMAL ENFORCEMENT ACTION, ICIS-NPD...",45.109000,-95.035210


In [12]:
# convert to json
final_epa_frs_clean.to_json('epa_frs_data.json', orient='records')
